# import

In [9]:
import os
import json
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# check ollama status

In [2]:
!curl --location 'http://127.0.0.1:11434/api/generate' \
--header 'Content-Type: application/json' \
--data '{ \
    "model": "llama3:8b", \
    "prompt": "why is the sky blue ?",  \
    "stream": false, \
    "options": { \
        "temperature": 0 \
    } \
}' \
# | python -m json.tool

{"model":"llama3:8b","created_at":"2024-04-26T18:57:35.616563335Z","response":"What a great question!\n\nThe short answer is: scattering of light by tiny molecules in the atmosphere.\n\nHere's a more detailed explanation:\n\nWhen sunlight enters Earth's atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of visible light, so they scatter the light in all directions. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century.\n\nNow, here's the important part: shorter (blue) wavelengths are scattered more than longer (red) wavelengths. This is because the smaller molecules are more effective at scattering the shorter wavelengths. Think of it like a game of pool: the smaller balls (blue light) bounce around more easily than the larger ones (red light).\n\nAs a result, our eyes perceive the blue light as being scatte

In [3]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"llama3:8b","model":"llama3:8b","modified_at":"2024-04-25T15:53:32.490960207+02:00","size":4661224578,"digest":"a6990ed6be412c6a217614b0ec8e9cd6800a743d5dd7e1d7fbe9df09e61d5615","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8B","quantization_level":"Q4_0"}}]}

# langchain with Ollama model

In [4]:
llm = Ollama(model="llama3:8b", temperature=0)

## testing llm: invoke


In [3]:
# llm.invoke(input="tell me a joke")
llm.invoke("how can langsmith help with testing ?")


"Langsmith, a language model, can assist with testing in several ways:\n\n1. **Automated Testing**: Langsmith can be used to generate test cases and test data for automated testing frameworks like Selenium or Pytest. It can create scenarios, inputs, and expected outputs based on the requirements.\n2. **Natural Language Processing (NLP) Testing**: Langsmith's NLP capabilities can help test text-based APIs, chatbots, or language processing systems. It can generate test cases that simulate user interactions, such as sending messages or making requests.\n3. **Error Detection**: Langsmith can be trained to detect errors in natural language inputs, such as syntax errors, grammatical errors, or semantic errors. This can help identify issues early on in the development process.\n4. **Test Data Generation**: Langsmith can generate test data for testing APIs, databases, or other systems that rely on text-based inputs. It can create a wide range of test cases, including edge cases and corner case

## testing llm model: chat prompt

In [4]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

chain = chat_prompt_template | llm

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'm excited to share some insights on how Langsmith can assist with testing.

Langsmith's AI-powered language processing capabilities can be leveraged in various ways to enhance the testing process. Here are some ideas:

1. **Automated Testing**: Langsmith can help automate testing by generating test cases based on natural language descriptions. This can save time and reduce the need for manual testing.
2. **Test Data Generation**: Langsmith's AI can generate test data, such as input values or expected output results, based on predefined patterns or scenarios. This can help ensure that tests cover a wide range of possible inputs and edge cases.
3. **Error Message Analysis**: When errors occur during testing, Langsmith can analyze error messages to identify the root cause and provide insights for debugging. This can save time and effort in identifying and resolving issues.
4. **Test Case Prioritization**: Langsmith's AI can help prioritiz

## testing llm model: chat prompt & StrOutputParser

In [5]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

output_parser = StrOutputParser()

chain = chat_prompt_template | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'm excited to share some insights on how Langsmith can assist with testing.

Langsmith's AI-powered language processing capabilities can be leveraged in various ways to enhance the testing process. Here are some ideas:

1. **Automated Testing**: Langsmith can help automate testing by generating test cases based on natural language descriptions. This can save time and reduce the likelihood of human error.
2. **Test Data Generation**: Langsmith's AI can generate test data, such as input values or expected output results, based on predefined patterns or scenarios. This can be particularly useful for testing complex systems or APIs.
3. **Error Detection**: Langsmith's language processing capabilities can help detect errors in test cases by analyzing the syntax and semantics of the test code. This can identify potential issues before they cause problems during testing.
4. **Test Case Prioritization**: Langsmith can analyze test cases based o

## retrieval chain

In [31]:
web_base_loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = web_base_loader.load()
# print(docs)
# print(type(docs))
# print(len(docs))
# type(docs[0])
# print(docs[0].page_content)

ollama_embedding = OllamaEmbeddings()

recursive_character_text_splitter = RecursiveCharacterTextSplitter()
documents = recursive_character_text_splitter.split_documents(documents=docs)
# print(type(documents))
# print(len(documents))
# print(documents[0])
# print(documents[3])